This works, with the model from AMAbot.ipynb on 5 Jun 2023 before 11:48am.

It works on a T4 (15GB GPU RAM). But only if you request the high-RAM machines! (otherwise you can't load the base model)

But.. you can't merge LoRA weights back into 8-bit models, and 16-bit model (just loading  without the 8-bit) can't be loaded on a T4.

In [1]:
!nvidia-smi

Tue Jun  6 00:22:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    43W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# # https://jaketae.github.io/study/gpt2/
# #  import locale
# #  locale.getpreferredencoding = lambda: "UTF-8"
!pip install transformers 
!pip install huggingface_hub

!pip install peft
!pip install accelerate
!pip install bitsandbytes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 92.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 29.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 20.0 MB/s eta 0:00:00


In [3]:
# from huggingface_hub import notebook_login
# notebook_login()
# raise ValueError

In [4]:
# import transformers
import torch
from transformers import (
            AutoTokenizer, AutoModelForCausalLM,
            DataCollatorForLanguageModeling,
            Trainer, TrainingArguments,
            get_cosine_schedule_with_warmup)
from dataclasses import dataclass
# import fire
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
)
from peft import prepare_model_for_int8_training
from transformers import Trainer

import os

device = 'cuda'


from peft import PeftConfig, PeftModel
# config = PeftConfig.from_pretrained(repo_name)


# Loading the base model
tokenizer_inf = AutoTokenizer.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-7B-v0.1", padding_side='left')
# base_model = AutoModelForCausalLM.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-7B-v0.1", device_map='auto', torch_dtype=torch.float16, load_in_8bit=True)
base_model = AutoModelForCausalLM.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-7B-v0.1", device_map='auto', torch_dtype=torch.float16) # Not loadinig in  8 bit beccause we're merging!


repo_name= 'AMAbot/AMAbot-7B'

# Loading LoRA  weights
model = PeftModel.from_pretrained(base_model, repo_name) 
model.eval()


model.gradient_checkpointing_disable() # Probably no effect for inference

print(tokenizer_inf.model_max_length)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('http'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2048


In [14]:
# prompt = "<human>: What is the sound of one hand clapping?\n<bot>:"
# prompt = "<human>: Q for tszzl: What is the sound of one hand clapping?\n<bot>:"
# prompt = "<human>: Q for FINALLEVEL: What is the sound of one hand clapping?\n<bot>:"
# prompt = "<human>: Q for KarpaT: What is your favorite programming language?\n<bot>:"
# prompt = "<human>: Q for KarpaT: What is your favorite video game?\n<bot>:"
# prompt = "<human>: Q for KarpaT: What are you doing right now?\n<bot>:"
# prompt="<human>: When was the term OG first introduced to the world?\n<bot>:"
# prompt="<human>: Q for FINALLEVEL: When was the term OG first introduced to the world?\n<bot>:"
# prompt = "<human>: Q for tszzl: How is it going?\n<bot>:"
# prompt = "<human>: Q for FINALLEVEL: How is it going?\n<bot>:"
# prompt = "<human>: How is it going?\n<bot>:"
# prompt = "My dream is"
prompts = [

    "<human>: Q for Karpathy: What is the sound of one hand clapping?\n<bot>:",  
    "<human>: Q for Karpathy: What is the sound of one hand clapping?\n<bot>:",  
    "<human>: Q for Karpathy: What is the sound of one hand clapping?\n<bot>:", 
    "<human>: Q for Karpathy: What is your favorite programming language?\n<bot>:",
    "<human>: Q for Karpathy: What is your favorite programming language?\n<bot>:",
    "<human>: Q for Karpathy: What is your favorite programming language?\n<bot>:",
    "<human>: Q for Karpathy: What are you doing right now?\n<bot>:",
    "<human>: Q for Karpathy: What are you doing right now?\n<bot>:",
    "<human>: Q for Karpathy: What are you doing right now?\n<bot>:",

    "<human>: Q for Ice-T: What is the sound of one hand clapping?\n<bot>:",  
    "<human>: Q for Ice-T: What is the sound of one hand clapping?\n<bot>:",  
    "<human>: Q for Ice-T: What is the sound of one hand clapping?\n<bot>:", 
    "<human>: Q for Ice-T: What is your favorite programming language?\n<bot>:",
    "<human>: Q for Ice-T: What is your favorite programming language?\n<bot>:",
    "<human>: Q for Ice-T: What is your favorite programming language?\n<bot>:",
    "<human>: Q for Ice-T: What are you doing right now?\n<bot>:",
    "<human>: Q for Ice-T: What are you doing right now?\n<bot>:",
    "<human>: Q for Ice-T: What are you doing right now?\n<bot>:",

    "<human>: Q for Karpa-T: What is the sound of one hand clapping?\n<bot>:",  
    "<human>: Q for Karpa-T: What is the sound of one hand clapping?\n<bot>:",  
    "<human>: Q for Karpa-T: What is the sound of one hand clapping?\n<bot>:", 
    "<human>: Q for Karpa-T: What is your favorite programming language?\n<bot>:",
    "<human>: Q for Karpa-T: What is your favorite programming language?\n<bot>:",
    "<human>: Q for Karpa-T: What is your favorite programming language?\n<bot>:",
    "<human>: Q for Karpa-T: What are you doing right now?\n<bot>:",
    "<human>: Q for Karpa-T: What are you doing right now?\n<bot>:",
    "<human>: Q for Karpa-T: What are you doing right now?\n<bot>:",

    ]



tokenizer_inf.pad_token = tokenizer_inf.eos_token 

inputs = tokenizer_inf(prompts, return_tensors='pt',  padding='longest').to(device)

print(inputs.input_ids.shape)
input_length = inputs.input_ids.shape[1]
outputs = model.generate(
    # **inputs, max_new_tokens=64, do_sample=True, temperature=0.7, top_p=0.7, top_k=50, return_dict_in_generate=True
    **inputs, max_new_tokens=64, do_sample=True, top_p=0.7, return_dict_in_generate=True
)
# tokenz = outputs.sequences[:, input_length:]  # If we want to remove the questions from the answers
tokenz = outputs.sequences  
output_list = tokenizer_inf.batch_decode(tokenz)
for x in output_list:
    print(x.replace('<|endoftext|>','')+"\n==" )

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


torch.Size([27, 24])
<human>: Q for Karpathy: What is the sound of one hand clapping?
<bot>: (Karpathy talking) The sound of one hand clapping is like a bomb going off in a crowded theater. 🤦‍♂️🤡🇺🇸😂🤔🤦‍♂️🤦‍�
==
<human>: Q for Karpathy: What is the sound of one hand clapping?
<bot>: (Karpathy talking) “The sound of one hand clapping” from the original “2001: A Space Odyssey” 🎤👏🏽👊🏽🔥🔥🔥🔥🔥🔥🔥🔥
==
<human>: Q for Karpathy: What is the sound of one hand clapping?
<bot>: (Karpathy talking) "The sound of one hand clapping" The video is an attempt at animating a sound wave, a bit of fun :D:D (on my part) + the soundtrack is the original. I'd like to get better at animating sound. The "sound of one hand cl
==
<human>: Q for Karpathy: What is your favorite programming language?
<bot>: (Karpathy talking) My favorite programming language is a language where you type in "Hello, World" and get an object back that has a "get" method. I don't know what it is, but it's that simple. Works with anything, even

In [7]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!nvidia-smi

Tue Jun  6 00:26:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    49W / 400W |  27097MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [15]:
model = model.merge_and_unload()
sizeinbytes = model.get_memory_footprint()
print(sizeinbytes, "bytes")
print(sizeinbytes // (2**20), "megabytes")

13848830080 bytes
13207 megabytes


In [12]:
if True:
    from huggingface_hub import notebook_login

    notebook_login()

In [16]:

if True:
    model.push_to_hub("AMAbot/AMAbotMerged-7B")


pytorch_model-00001-of-00002.bin:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.80G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
if True:
    tokenizer_inf.push_to_hub("AMAbot/AMAbotMerged-7B")
